In [1]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# The list of 20 type of cryptocurrency

coin_list = ['BTC', 'ETH', 'XRP', 'ADA', 'USDT', 'DOGE', 'XLM', 'DOT', 'UNI', 'UNI', 'LINK', 'USDC' 'BCH' 'LTC', 'GRT', 'ETC', 'FIL', 'AAVE', 'ALGO', 'EOS']

In [3]:
# Define the dataframe

main_df = pd.DataFrame()

In [6]:
for coin in coin_list:
    coin_df = pd.DataFrame()
    df = pd.DataFrame(index=[0])
    
    # Defining the Start Date and End Date
    datetime_end = datetime(2021, 7, 2, 0, 0)
    datetime_check = datetime(2021, 7, 1, 0, 0)
    
    while len(df) > 0:
        if datetime_end == datetime_check:
            break
        
        datetime_start = datetime_end - relativedelta(hours = 12)
        
        #Api for the scrapping
        url = 'https://production.api.coindesk.com/v2/price/values/'+ coin +'?start_date='+datetime_start.strftime("%Y-%m-%dT%H:%M") + '&end_date=' + datetime_end.strftime("%Y-%m-%dT%H:%M") + '&ohlc=true'
        
        #we are using the request to fetch the data from the api in the json format and then storing it into the dataframe.
        temp_data = requests.get(url).json()
        df = pd.DataFrame(temp_data['data']['entries'])
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        
        # To handle the Missing Data
        insert_ids_list = [np.nan]
        
        '''In numerous occasions, I observed that CoinDesk did not collect every minute of data. 
        According to observation, in a normal circumstance, the timestamp would have a discrepancy
        value of 60000 for 1 minute. As a result, I can quickly tell that the missing period is the
        time gap if the row difference for the timestamp is larger than 60000. To deal with it, a
        hot-deck imputation procedure is used. To put it another way, the nearest minute data will 
        be substituted for the missing one.
        '''
        
        while len(insert_ids_list) > 0:
            timestamp_checking = np.array(df['Timestamp'][1:]) - np.array(df['Timestamp'][:-1])
            insert_ids_list = np.where(timestamp_checking!= 60000)[0]
            if len(insert_ids_list) > 0:
                print(str(len(insert_ids_list)) + ' mismatched.')
                insert_ids = insert_ids_list[0]
                temp_df = df.iloc[insert_ids.repeat(int(timestamp_checking[insert_ids]/60000)-1)].reset_index(drop=True)
                temp_df['Timestamp'] = [temp_df['Timestamp'][0] + i*60000 for i in range(1, len(temp_df)+1)]
                df = df.loc[:insert_ids].append([temp_df]).append(df.loc[insert_ids+1:]).reset_index(drop=True)
                insert_ids_list = insert_ids_list[1:]
                
        
        #adding datetime and symbol to dataframe
        df = df.drop(['Timestamp'], axis=1)
        df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]
        coin_df = df.append([coin_df])
        datetime_end = datetime_start
        
    coin_df['Symbol'] = coin
    main_df = main_df.append([coin_df])

/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append([coin_df])


1 mismatched.


/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.loc[:insert_ids].append([temp_df]).append(df.loc[insert_ids+1:]).reset_index(drop=True)
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append([coin_df])
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append([coin_df])
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

1 mismatched.


/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.loc[:insert_ids].append([temp_df]).append(df.loc[insert_ids+1:]).reset_index(drop=True)
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append([coin_df])
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append([coin_df])
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

1 mismatched.


/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append([coin_df])


1 mismatched.


/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.loc[:insert_ids].append([temp_df]).append(df.loc[insert_ids+1:]).reset_index(drop=True)
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coin_df = df.append([coin_df])
/var/folders/s5/qk1q_v9s245f9h_m0zc4b0v00000gn/T/ipykernel_32450/1616070760.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df = main_df.append([coin_df])


KeyError: 'data'